In [34]:
from dataset import TensorflowDataset
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
# 

In [21]:
test_data = pd.read_pickle('extracted_features\\features_test.pkl')
test = tf.data.Dataset.from_tensor_slices((test_data, np.zeros(len(test_data))))

In [22]:
test_silence_data = pd.read_pickle('extracted_features\\features_test_silence.pkl')
test_silence = tf.data.Dataset.from_tensor_slices((test_silence_data, np.zeros(len(test_silence_data))))

In [23]:
import numpy as np
import models
from models import Lstm, Gru, Transformer, TransformerBlock
from dataset import LABELS
import pandas as pd

class final_model:
    def __init__(self, model, test, test_silence):

        self.model = model
        self.test = test
        self.test_silence = test_silence

        LABELS_dict = {i:LABELS[i] for i in range(len(LABELS))}
        LABELS_dict[10] = 'unknown'
        self.LABELS_dict = LABELS_dict

        self.predicted_labels = None
        self.silence_labels = None

    def predict_silence(self, model_path='models\\lstm_silence.h5'):
        model_sil = tf.keras.models.load_model(model_path)
        self.silence_labels = np.argmax(model_sil.predict(self.test.batch(32)), axis=1)
    
    def predict_label(self):
        self.predicted_labels = self.model.predict(self.test)

    def save_results(self, submission_name=f'{}'):
        x = pd.concat([pd.DataFrame(self.predicted_labels), pd.DataFrame(self.silence_labels)], axis=1)
        x.columns = ['label', 'silence']
        res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: self.LABELS_dict[x]))
        res = pd.DataFrame(res)
        res.columns = ['label_prediction']
        res.to_csv(f'results\\submission_{submission_name}.csv', index=False, header=True)

In [33]:
model = Gru(from_path = 'models\\gru_mod.h5')
model2 = Transformer(from_path = 'models\\transformer_mod.h5')
d = final_model(model, test, test_silence)

ValueError: Unknown layer: 'TransformerBlock'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [24]:
d.predict_silence()

In [25]:
d.predict_label()

   57/15854 [..............................] - ETA: 16:41

KeyboardInterrupt: 

In [32]:
print('Number of silences:', len(d.test) - d.silence_labels.sum())
print('Total number of observations:', len(d.test))

Number of silences: 801
Total number of observations: 158538


In [ ]:
d.save_results()

In [33]:

class ensamble_model:
    def __init__(self):
        # train = pd.read_pickle('features_test_silence.pkl')
        # y_train = np.array([x[1] for x in train])
        # labels = list(np.unique(y_train))
        # self.labels_silence = labels
        # self.silence_det = TensorflowDataset('features_test_silence.pkl', labels=self.labels_silence).dataset
        
        train = pd.read_pickle('features_test.pkl')
        y_train = np.array([x[1] for x in train])
        labels = list(np.unique(y_train))
        self.labels_label = labels
        self.label_det = TensorflowDataset('features_test.pkl', labels=self.labels_label).dataset
    
        self.silence_model = tf.keras.models.load_model('models\\silence_model.h5')
        self.final_model = 'dupa'#tf.keras.models.load_model('models\\final_model.h5')
    
    def predict_silence(self):
        pred = self.silence_model.predict(self.silence_det)


    def save():
        # otwórz plik csv do zapisu
        with open('predictions.csv', mode='w', newline='') as file:
            # utwórz obiekt writer i ustaw separator na przecinek
            writer = csv.writer(file, delimiter=',')
            
            # zapisz nagłówek
            writer.writerow(['col1', 'col2', 'col3', 'col4', 'prediction'])
            
            # zapisz dane i przewidywania
            for i in range(len(data)):
                row = data[i] + [predictions[i]]
                writer.writerow(row)



In [34]:
model = ensamble_model()

TypeError: unhashable type: 'numpy.ndarray'